# TOI-5807.01 Statistical Validation Walkthrough (TIC 188646744)

This notebook is a **step-by-step validation walkthrough** for a real TESS planet candidate. It is written for astronomers who want to understand *why* each diagnostic is run and what evidence it provides.

It is intended to be the primary end-to-end tutorial for TOI-5807.01, replacing older notebooks over time.

Key principles:

- **Traceable**: each step prints compact, machine-readable summaries.
- **Reproducible**: uses a bundled dataset folder and explicit toggles.
- **Domain-first**: we use astrophysics language (odd/even, secondary eclipse, centroid shifts, dilution, AO constraints) rather than internal check IDs.

You can run this notebook top-to-bottom, or jump directly to the sections you care about.


## Target overview

**TOI-5807.01** (TIC 188646744; HD 196216) is a shallow transit candidate observed by TESS in multiple sectors. The goal here is **statistical validation**: demonstrate that the signal is extremely unlikely to be a false positive and is consistent with an on-target transiting planet.

Evidence categories we will collect:

1. **Light-curve morphology**: planet-like transit shape, no odd/even mismatch, no significant secondary eclipse.
2. **Instrumental/systematics robustness**: consistency across sectors; diagnostics for non-transit phase structure.
3. **On-target localization**: pixel-level centroid/difference-image localization (when per-sector TPFs are available).
4. **Blend constraints**: AO contrast curve reduces probability of unresolved companions.
5. **False Positive Probability (FPP)**: TRICERATOPS(+), incorporating AO constraints.


In [1]:
from __future__ import annotations

import json
import tempfile
import warnings
from pathlib import Path

import numpy as np

import bittr_tess_vetter.api as btv

# Optional network utilities
from bittr_tess_vetter.api.catalogs import fetch_exofop_toi_table
from bittr_tess_vetter.api.fpp import calculate_fpp

# Reduce noisy, non-scientific warnings in notebook outputs (improves traceability)
warnings.filterwarnings(
    "ignore",
    message=r"Warning: the tpfmodel submodule is not available without oktopus installed",
)
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module=r"lightkurve\\.prf",
)
warnings.filterwarnings(
    "ignore",
    message=r"IProgress not found\. Please update jupyter and ipywidgets\.",
)
warnings.filterwarnings(
    "ignore",
    message=r"pkg_resources is deprecated as an API\.",
)

# ----------------------------------------------------------------------------
# Controls (edit these)
# ----------------------------------------------------------------------------
NETWORK = True               # enables catalog lookups and FPP upstream calls

# Keep analysis inputs stable by default. ExoFOP values can change over time.
USE_EXOFOP_PARAMETERS = False

RUN_FPP = True               # set False to skip (can take minutes)
RUN_PIXEL_VETTING = True     # set False to skip (needs lightkurve + network)

# FPP reproducibility knobs (only used when RUN_FPP=True)
FPP_PRESET = "fast"          # "fast" or "standard"
FPP_REPLICATES = 5
FPP_SEED = 42
FPP_TIMEOUT_SECONDS = 1800

TIC_ID = 188646744
TOI_LABEL = "TOI-5807.01"

# Coordinates for catalog queries when NETWORK=True
# (These are only used for catalog-style checks; they do not affect LC-only diagnostics.)
RA_DEG = 304.12005
DEC_DEG = 11.08344


<details>
<summary><b>Expected Output</b></summary>

No output is expected for the imports cell.
</details>


## Run configuration (traceability)

This cell prints the key toggles and library versions. If someone reproduces this notebook, this is the first place to compare environments.


In [2]:
import importlib.metadata

def _pkg_version(name: str) -> str | None:
    try:
        return importlib.metadata.version(name)
    except Exception:
        return None

print(
    {
        "NETWORK": NETWORK,
        "RUN_FPP": RUN_FPP,
        "RUN_PIXEL_VETTING": RUN_PIXEL_VETTING,
        "FPP_PRESET": FPP_PRESET,
        "FPP_REPLICATES": FPP_REPLICATES,
        "FPP_SEED": FPP_SEED,
        "FPP_TIMEOUT_SECONDS": FPP_TIMEOUT_SECONDS,
        "versions": {
            "bittr-tess-vetter": _pkg_version("bittr-tess-vetter"),
            "numpy": _pkg_version("numpy"),
            "astropy": _pkg_version("astropy"),
            "wotan": _pkg_version("wotan"),
            "lightkurve": _pkg_version("lightkurve"),
        },
    }
)


{'NETWORK': True, 'RUN_FPP': True, 'RUN_PIXEL_VETTING': True, 'FPP_PRESET': 'fast', 'FPP_REPLICATES': 5, 'FPP_SEED': 42, 'FPP_TIMEOUT_SECONDS': 1800, 'versions': {'bittr-tess-vetter': '0.1.0', 'numpy': '2.3.5', 'astropy': '7.2.0', 'wotan': '1.10', 'lightkurve': '2.5.1'}}


<details>
<summary><b>Expected Output</b></summary>

```text
{'NETWORK': True, 'RUN_FPP': True, 'RUN_PIXEL_VETTING': True, 'FPP_PRESET': 'fast', 'FPP_REPLICATES': 5, 'FPP_SEED': 42, 'FPP_TIMEOUT_SECONDS': 1800, 'versions': {'bittr-tess-vetter': '0.1.0', 'numpy': '2.3.5', 'astropy': '7.2.0', 'wotan': '1.10', 'lightkurve': '2.5.1'}}
```

</details>


## 1) Load the analysis dataset

We use the bundled tutorial dataset for TOI-5807.01. This makes the workflow runnable even without downloading light curves.

The dataset includes per-sector light curves and a PHARO AO contrast curve.


In [3]:
ds = btv.load_tutorial_target("tic188646744")
ds.summary()

{'schema_version': 1,
 'root': '/Users/collier/projects/apps/bittr-tess-vetter/docs/tutorials/data/tic188646744',
 'sectors_lc': [55, 75, 82, 83],
 'sectors_tpf': [83],
 'artifacts': ['files']}

<details>
<summary><b>Expected Output</b></summary>

```text
{'schema_version': 1,
 'root': '/Users/collier/projects/apps/bittr-tess-vetter/docs/tutorials/data/tic188646744',
 'sectors_lc': [55, 75, 82, 83],
 'sectors_tpf': [83],
 'artifacts': ['files']}
```

</details>


## 2) Candidate ephemeris and stellar context

We pull ExoFOP values when available (NETWORK on). Otherwise we use stable fallback values.

We keep the ephemeris explicit because nearly all diagnostics depend on it.


In [4]:
def _safe_float(x, default=None):
    try:
        return float(x)
    except Exception:
        return default


toi = None
if NETWORK:
    toi_table = fetch_exofop_toi_table()
    matches = toi_table.entries_for_tic(TIC_ID)
    toi = matches[0] if matches else None

# Use ExoFOP parameters only when explicitly requested.
# Default behavior is stable/tutorial-reproducible inputs.
if USE_EXOFOP_PARAMETERS and toi is not None:
    PERIOD_DAYS = _safe_float(toi.get("period_days"), 14.2423724)
    epoch_bjd = _safe_float(toi.get("epoch_bjd"), 2460540.26317)
    T0_BTJD = (epoch_bjd - 2457000.0) if epoch_bjd and epoch_bjd > 2_450_000 else (epoch_bjd or 3540.26317)
    DURATION_HOURS = _safe_float(toi.get("duration_hours"), 4.046)
else:
    PERIOD_DAYS = 14.2423724
    T0_BTJD = 3540.26317
    DURATION_HOURS = 4.046

# Measure an on-the-fly depth estimate from the stitched light curve (deterministic).
stitched = btv.stitch_lightcurves(
    [
        {
            "time": np.asarray(lc.time, dtype=np.float64),
            "flux": np.asarray(lc.flux, dtype=np.float64),
            "flux_err": np.asarray(lc.flux_err, dtype=np.float64),
            "sector": int(sector),
            "quality": np.zeros(len(lc.time), dtype=np.int32),
        }
        for sector, lc in sorted(ds.lc_by_sector.items())
    ]
)
in_tr = btv.get_in_transit_mask(stitched.time, PERIOD_DAYS, T0_BTJD, DURATION_HOURS)
out_tr = btv.get_out_of_transit_mask(
    stitched.time,
    PERIOD_DAYS,
    T0_BTJD,
    DURATION_HOURS,
    buffer_factor=3.0,
)
depth_hat, depth_err = btv.measure_transit_depth(stitched.flux, in_tr, out_tr)
DEPTH_PPM = float(depth_hat * 1e6)
DEPTH_ERR_PPM = float(depth_err * 1e6)

stellar = btv.StellarParams(
    radius=_safe_float(toi.get("stellar_radius_r_sun"), 1.65) if toi else 1.65,
    mass=_safe_float(toi.get("stellar_mass_m_sun"), 1.47) if toi else 1.47,
    teff=_safe_float(toi.get("stellar_eff_temp_k"), 6816.0) if toi else 6816.0,
    logg=_safe_float(toi.get("stellar_logg"), 4.17) if toi else 4.17,
)

ephem = btv.Ephemeris(period_days=PERIOD_DAYS, t0_btjd=T0_BTJD, duration_hours=DURATION_HOURS)
candidate = btv.Candidate(ephemeris=ephem, depth_ppm=DEPTH_PPM)

print(
    {
        "period_days": ephem.period_days,
        "t0_btjd": ephem.t0_btjd,
        "duration_hours": ephem.duration_hours,
        "depth_ppm": candidate.depth_ppm,
        "depth_err_ppm": DEPTH_ERR_PPM,
        "exofop_depth_ppm": None if toi is None else _safe_float(toi.get("depth_ppm")),
        "stellar_radius_rsun": stellar.radius,
        "stellar_mass_msun": stellar.mass,
    }
)


{'period_days': 14.2423724, 't0_btjd': 3540.26317, 'duration_hours': 4.046, 'depth_ppm': 255.55864733783906, 'depth_err_ppm': 8.445013682052277, 'exofop_depth_ppm': 225.0, 'stellar_radius_rsun': 1.65, 'stellar_mass_msun': 1.47}


<details>
<summary><b>Expected Output</b></summary>

```text
{'period_days': 14.2423724, 't0_btjd': 3540.26317, 'duration_hours': 4.046, 'depth_ppm': 255.55864733783906, 'depth_err_ppm': 8.445013682052277, 'exofop_depth_ppm': 225.0, 'stellar_radius_rsun': 1.65, 'stellar_mass_msun': 1.47}
```

</details>


## 3) Light-curve evidence (planet-like morphology)

We now run the core light-curve vetting checks on a stitched multi-sector light curve:

- **Odd/even test**: an eclipsing binary at half the reported period can show alternating depths.
- **Secondary eclipse search**: a significant eclipse near phase 0.5 is a red flag for an EB.
- **Transit shape / duration / depth stability**: broad consistency checks that the signal behaves like a transit.

These diagnostics should be interpreted as evidence; they are not a planet mass measurement.


In [5]:
wf_default = btv.run_candidate_workflow(
    lc_by_sector=ds.lc_by_sector,
    candidate=candidate,
    stellar=stellar,
    preset="default",
    network=NETWORK,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    run_per_sector=True,
)

print(btv.format_vetting_table(wf_default.bundle))

r01 = wf_default.bundle.get_result("V01")
r02 = wf_default.bundle.get_result("V02")

summary = {
    "counts": {
        "checks": len(wf_default.bundle.results),
        "ok": wf_default.bundle.n_passed,
        "error": wf_default.bundle.n_failed,
        "skipped": wf_default.bundle.n_unknown,
    },
    "key_metrics": {
        "odd_even_delta_sigma": None if r01 is None else r01.metrics.get("delta_sigma"),
        "secondary_depth_sigma": None if r02 is None else r02.metrics.get("secondary_depth_sigma"),
    },
    "per_sector_summary": wf_default.per_sector.summary_records if wf_default.per_sector else None,
}
print(json.dumps(summary, indent=2, sort_keys=True))


Vetting Results
Checks: 15  ok: 12  error: 0  skipped: 3

ID    Name                  Status   Confidence
----  --------------------  -------  ----------
V01   Odd-Even Depth        ok       0.700
V02   Secondary Eclipse     ok       0.850
V03   Duration Consistency  ok       0.850
V04   Depth Stability       ok       0.700
V05   V-Shape               ok       0.935
V06   Nearby EB Search      ok       0.600
V07   ExoFOP TOI Lookup     ok       0.800
V08   Centroid Shift        skipped
V09   Difference Image      skipped
V10   Aperture Dependence   skipped
V11   ModShift              ok       1.000
V11b  ModShiftUniqueness    ok       0.900
V12   SWEET                 ok       1.000
V13   Data Gaps             ok       0.750
V15   Transit Asymmetry     ok       0.750

{
  "counts": {
    "checks": 15,
    "error": 0,
    "ok": 12,
    "skipped": 3
  },
  "key_metrics": {
    "odd_even_delta_sigma": 1.73,
    "secondary_depth_sigma": 1.12
  },
  "per_sector_summary": [
    {
      "chec

<details>
<summary><b>Expected Output</b></summary>

```text
Vetting Results
============================================================
Checks: 15  ok: 12  error: 0  skipped: 3

ID    Name                  Status   Confidence
----  --------------------  -------  ----------
V01   Odd-Even Depth        ok       0.700
V02   Secondary Eclipse     ok       0.850
V03   Duration Consistency  ok       0.850
V04   Depth Stability       ok       0.700
V05   V-Shape               ok       0.935
V06   Nearby EB Search      ok       0.600
V07   ExoFOP TOI Lookup     ok       0.800
V08   Centroid Shift        skipped
V09   Difference Image      skipped
V10   Aperture Dependence   skipped
V11   ModShift              ok       1.000
V11b  ModShiftUniqueness    ok       0.900
V12   SWEET                 ok       1.000
V13   Data Gaps             ok       0.750
V15   Transit Asymmetry     ok       0.750

{
  "counts": {
    "checks": 15,
    "error": 0,
    "ok": 12,
    "skipped": 3
  },
  "key_metrics": {
    "odd_even_delta_sigma": 1.73,
    "secondary_depth_sigma": 1.12
  },
  "per_sector_summary": [
    {
      "checks": 15,
      "error": 0,
      "ok": 12,
      "sector": 55,
      "skipped": 3
    },
    {
      "checks": 15,
      "error": 0,
      "ok": 12,
      "sector": 75,
      "skipped": 3
    },
    {
      "checks": 15,
      "error": 0,
      "ok": 12,
      "sector": 82,
      "skipped": 3
    },
    {
      "checks": 15,
      "error": 0,
      "ok": 12,
      "sector": 83,
      "skipped": 3
    }
  ]
}
```

</details>


## 4) Robustness and additional diagnostics

Here we run optional metrics-only diagnostics designed to prompt further inspection:

- **Model competition**: does a transit-only model compete well against simple non-transit alternatives?
- **Alias/phase diagnostics**: does the phased light curve show strong non-transit events at other phases?
- **Per-sector ephemeris metrics**: are depth and SNR-like metrics consistent by sector?


In [6]:
wf_ext = btv.run_candidate_workflow(
    lc_by_sector=ds.lc_by_sector,
    candidate=candidate,
    stellar=stellar,
    preset="extended",
    network=NETWORK,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    run_per_sector=True,
)

r16 = wf_ext.bundle.get_result("V16")
r19 = wf_ext.bundle.get_result("V19")

def _r(x: float | None, nd: int) -> float | None:
    if x is None:
        return None
    return float(np.round(float(x), nd))

sector_metrics = []
for m in (wf_ext.per_sector.sector_ephemeris_metrics if wf_ext.per_sector else []):
    d = m.to_dict()
    sector_metrics.append(
        {
            "sector": int(d.get("sector")),
            "n_in_transit": int(d.get("n_in_transit")),
            "depth_hat_ppm": _r(d.get("depth_hat_ppm"), 1),
            "depth_sigma_ppm": _r(d.get("depth_sigma_ppm"), 1),
            "score": _r(d.get("score"), 2),
        }
    )

summary = {
    "V16_model_competition": None if r16 is None else {
        "winner": r16.metrics.get("winner"),
        "label": r16.metrics.get("model_competition_label"),
        "winner_margin_bic": _r(r16.metrics.get("winner_margin_bic"), 2),
        "artifact_risk": _r(r16.metrics.get("artifact_risk"), 2),
    },
    "V19_phase_diagnostics": None if r19 is None else {
        "n_phase_shift_events": int(r19.metrics.get("n_phase_shift_events")),
        "max_phase_shift_event_sigma": _r(r19.metrics.get("max_phase_shift_event_sigma"), 2),
        "secondary_significance_sigma": _r(r19.metrics.get("secondary_significance_sigma"), 2),
    },
    "sector_ephemeris_metrics": sector_metrics,
}
print(json.dumps(summary, indent=2, sort_keys=True))


{
  "V16_model_competition": {
    "artifact_risk": 0.8,
    "label": "SINUSOID",
    "winner": "transit_sinusoid",
    "winner_margin_bic": 11.19
  },
  "V19_phase_diagnostics": {
    "max_phase_shift_event_sigma": 16.09,
    "n_phase_shift_events": 5,
    "secondary_significance_sigma": 0.0
  },
  "sector_ephemeris_metrics": [
    {
      "depth_hat_ppm": 169.4,
      "depth_sigma_ppm": 13.4,
      "n_in_transit": 243,
      "score": 12.66,
      "sector": 55
    },
    {
      "depth_hat_ppm": 308.6,
      "depth_sigma_ppm": 13.6,
      "n_in_transit": 242,
      "score": 22.67,
      "sector": 75
    },
    {
      "depth_hat_ppm": 225.2,
      "depth_sigma_ppm": 13.4,
      "n_in_transit": 243,
      "score": 16.82,
      "sector": 82
    },
    {
      "depth_hat_ppm": 309.3,
      "depth_sigma_ppm": 13.4,
      "n_in_transit": 244,
      "score": 23.02,
      "sector": 83
    }
  ]
}


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "V16_model_competition": {
    "artifact_risk": 0.8,
    "label": "SINUSOID",
    "winner": "transit_sinusoid",
    "winner_margin_bic": 11.19
  },
  "V19_phase_diagnostics": {
    "max_phase_shift_event_sigma": 16.09,
    "n_phase_shift_events": 5,
    "secondary_significance_sigma": 0.0
  },
  "sector_ephemeris_metrics": [
    {
      "depth_hat_ppm": 169.4,
      "depth_sigma_ppm": 13.4,
      "n_in_transit": 243,
      "score": 12.66,
      "sector": 55
    },
    {
      "depth_hat_ppm": 308.6,
      "depth_sigma_ppm": 13.6,
      "n_in_transit": 242,
      "score": 22.67,
      "sector": 75
    },
    {
      "depth_hat_ppm": 225.2,
      "depth_sigma_ppm": 13.4,
      "n_in_transit": 243,
      "score": 16.82,
      "sector": 82
    },
    {
      "depth_hat_ppm": 309.3,
      "depth_sigma_ppm": 13.4,
      "n_in_transit": 244,
      "score": 23.02,
      "sector": 83
    }
  ]
}
```

</details>


## 5) On-target localization (pixel-level)

A major false-positive mode for TESS is a **blended eclipsing binary**: the transit-like signal originates from a nearby source within the photometric aperture.

Pixel-level vetting uses per-sector Target Pixel Files (TPFs) to check whether the in-transit centroid / difference image points to the target location.

The bundled dataset contains only one sector TPF stamp. To do this properly across sectors, we download per-sector TPFs.


In [7]:
if not RUN_PIXEL_VETTING:
    print("Skipping pixel-level vetting (set RUN_PIXEL_VETTING=True).")
else:
    if not NETWORK:
        raise ValueError("Set NETWORK=True to download per-sector TPFs")
    import lightkurve as lk
    import numpy as _np

    tpf_cache_dir = Path("persistent_cache/toi5807_tpfs")
    tpf_cache_dir.mkdir(parents=True, exist_ok=True)

    tpf_by_sector: dict[int, btv.TPFStamp] = {int(k): v for k, v in ds.tpf_by_sector.items()}
    for sector in sorted(ds.lc_by_sector.keys()):
        sector = int(sector)
        if sector in tpf_by_sector:
            continue

        npz_path = tpf_cache_dir / f"sector{sector}_tpf.npz"
        if npz_path.exists():
            d = _np.load(npz_path, allow_pickle=True)
            try:
                from astropy.wcs import WCS

                wcs = WCS(d["wcs_header"].item()) if "wcs_header" in d else None
            except Exception:
                wcs = d["wcs_header"].item() if "wcs_header" in d else None

            tpf_by_sector[sector] = btv.TPFStamp(
                time=_np.asarray(d["time"], dtype=_np.float64),
                flux=_np.asarray(d["flux"], dtype=_np.float64),
                flux_err=_np.asarray(d["flux_err"], dtype=_np.float64) if "flux_err" in d else None,
                wcs=wcs,
                aperture_mask=_np.asarray(d["aperture_mask"], dtype=bool) if "aperture_mask" in d else None,
                quality=_np.asarray(d["quality"], dtype=_np.int32) if "quality" in d else None,
            )
            continue

        search = lk.search_targetpixelfile(f"TIC {TIC_ID}", sector=int(sector), exptime=120)
        if len(search) == 0:
            print(f"No TPF for sector {sector}")
            continue
        tpf = search.download()
        if tpf is None:
            print(f"Download failed for sector {sector}")
            continue

        stamp = btv.TPFStamp(
            time=_np.asarray(tpf.time.value, dtype=_np.float64),
            flux=_np.asarray(tpf.flux.value, dtype=_np.float64),
            flux_err=_np.asarray(tpf.flux_err.value, dtype=_np.float64),
            wcs=tpf.wcs,
            aperture_mask=_np.asarray(tpf.pipeline_mask, dtype=bool),
            quality=_np.asarray(tpf.quality, dtype=_np.int32),
        )
        tpf_by_sector[sector] = stamp

        # Cache to disk so reruns do not need MAST downloads.
        try:
            wcs_header = dict(tpf.wcs.to_header()) if getattr(tpf, "wcs", None) is not None else None
            _np.savez(
                npz_path,
                time=stamp.time,
                flux=stamp.flux,
                flux_err=stamp.flux_err,
                wcs_header=wcs_header,
                aperture_mask=stamp.aperture_mask,
                quality=stamp.quality,
            )
        except Exception:
            pass

    per = btv.per_sector_vet(
        ds.lc_by_sector,
        candidate,
        stellar=stellar,
        tpf_by_sector=tpf_by_sector,
        preset="extended",
        network=False,
    )

    for sector, bundle in sorted(per.bundles_by_sector.items()):
        r_centroid = bundle.get_result("V08")
        r_diffimg = bundle.get_result("V09")
        r_apdep = bundle.get_result("V10")
        print(
            json.dumps(
                {
                    "sector": int(sector),
                    "centroid_shift_status": None if r_centroid is None else r_centroid.status,
                    "difference_image_status": None if r_diffimg is None else r_diffimg.status,
                    "aperture_dependence_status": None if r_apdep is None else r_apdep.status,
                },
                sort_keys=True,
            )
        )


{"aperture_dependence_status": "ok", "centroid_shift_status": "ok", "difference_image_status": "ok", "sector": 55}
{"aperture_dependence_status": "ok", "centroid_shift_status": "ok", "difference_image_status": "ok", "sector": 75}
{"aperture_dependence_status": "ok", "centroid_shift_status": "ok", "difference_image_status": "ok", "sector": 82}
{"aperture_dependence_status": "ok", "centroid_shift_status": "ok", "difference_image_status": "ok", "sector": 83}


<details>
<summary><b>Expected Output</b></summary>

```text
{"aperture_dependence_status": "ok", "centroid_shift_status": "ok", "difference_image_status": "ok", "sector": 55}
{"aperture_dependence_status": "ok", "centroid_shift_status": "ok", "difference_image_status": "ok", "sector": 75}
{"aperture_dependence_status": "ok", "centroid_shift_status": "ok", "difference_image_status": "ok", "sector": 82}
{"aperture_dependence_status": "ok", "centroid_shift_status": "ok", "difference_image_status": "ok", "sector": 83}
```

</details>


## 6) AO-assisted False Positive Probability (FPP)

Statistical validation typically uses **FPP < 1%** as a community convention. Here we compute TRICERATOPS(+), incorporating the PHARO AO contrast curve to rule out unresolved companions.

This is the most time-consuming step and involves upstream models (Gaia + TRILEGAL). We make it as reproducible as practical by setting an explicit seed.


In [8]:
if not RUN_FPP:
    print("Skipping FPP (set RUN_FPP=True).")
else:
    if not NETWORK:
        raise ValueError("Set NETWORK=True for FPP")

    cc = btv.load_contrast_curve_exofop_tbl(ds.root / "PHARO_Kcont_plot.tbl", filter="Kcont")
    import contextlib

    cache_dir = Path("persistent_cache/toi5807_fpp_cache")
    cache_dir.mkdir(parents=True, exist_ok=True)
    cache = btv.hydrate_cache_from_dataset(
        dataset=ds,
        tic_id=TIC_ID,
        flux_type="pdcsap",
        cache_dir=cache_dir,
        cadence_seconds=120.0,
    )

    log_path = Path("persistent_cache/toi5807_fpp_log.txt")
    with log_path.open("w") as f, contextlib.redirect_stdout(f), contextlib.redirect_stderr(f):
        fpp = calculate_fpp(
            cache=cache,
            tic_id=TIC_ID,
            period=ephem.period_days,
            t0=ephem.t0_btjd,
            depth_ppm=float(candidate.depth_ppm or DEPTH_PPM),
            duration_hours=ephem.duration_hours,
            sectors=sorted(ds.lc_by_sector.keys()),
            stellar_radius=stellar.radius,
            stellar_mass=stellar.mass,
            preset=FPP_PRESET,
            timeout_seconds=FPP_TIMEOUT_SECONDS,
            replicates=FPP_REPLICATES,
            seed=FPP_SEED,
            contrast_curve=cc,
        )

    if "error" in fpp:
        print(json.dumps(fpp, indent=2, sort_keys=True))
    else:
        summary = {
            "fpp": fpp.get("fpp"),
            "nfpp": fpp.get("nfpp"),
            "disposition": fpp.get("disposition"),
            "prob_planet": fpp.get("prob_planet"),
            "prob_eb": fpp.get("prob_eb"),
            "prob_beb": fpp.get("prob_beb"),
            "prob_neb": fpp.get("prob_neb"),
            "prob_ntp": fpp.get("prob_ntp"),
            "n_nearby_sources": fpp.get("n_nearby_sources"),
            "brightest_contaminant_dmag": fpp.get("brightest_contaminant_dmag"),
            "sectors_used": fpp.get("sectors_used"),
            "runtime_seconds": fpp.get("runtime_seconds"),
        }
        print(json.dumps(summary, indent=2, sort_keys=True))
        print(json.dumps({"fpp_log": str(log_path)}, indent=2))


{
  "brightest_contaminant_dmag": null,
  "disposition": "VALIDATED",
  "fpp": 1.7e-05,
  "n_nearby_sources": 347,
  "nfpp": 0.0,
  "prob_beb": 0.0,
  "prob_eb": 0.0,
  "prob_neb": 0.0,
  "prob_ntp": 0.0,
  "prob_planet": 0.976146,
  "runtime_seconds": 230.2,
  "sectors_used": [
    55,
    75,
    82,
    83
  ]
}
{
  "fpp_log": "persistent_cache/toi5807_fpp_log.txt"
}


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "brightest_contaminant_dmag": null,
  "disposition": "VALIDATED",
  "fpp": 1.7e-05,
  "n_nearby_sources": 347,
  "nfpp": 0.0,
  "prob_beb": 0.0,
  "prob_eb": 0.0,
  "prob_neb": 0.0,
  "prob_ntp": 0.0,
  "prob_planet": 0.976146,
  "runtime_seconds": 291.7,
  "sectors_used": [
    55,
    75,
    82,
    83
  ]
}
{
  "fpp_log": "persistent_cache/toi5807_fpp_log.txt"
}
```

</details>


## 7) Export a shareable report

We export a markdown report (for human review) and a JSON bundle (for programmatic auditing).


In [9]:
out_dir = Path("persistent_cache/toi5807_trace")
out_dir.mkdir(parents=True, exist_ok=True)

btv.export_bundle(wf_ext.bundle, format="md", path=out_dir / "report.md", title=f"{TOI_LABEL} ({TIC_ID})")
btv.export_bundle(wf_ext.bundle, format="json", path=out_dir / "bundle.json")

print(json.dumps({"out_dir": str(out_dir), "files": sorted(p.name for p in out_dir.iterdir())}, indent=2, sort_keys=True))


{
  "files": [
    "bundle.json",
    "report.md"
  ],
  "out_dir": "persistent_cache/toi5807_trace"
}


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "files": [
    "bundle.json",
    "report.md"
  ],
  "out_dir": "persistent_cache/toi5807_trace"
}
```

</details>


## Appendix: mapping to implementation checks

Internally, the library runs a set of named checks. You do not need to memorize IDs to use this tutorial, but if you want an audit trail:

- Odd/even depth consistency → `V01`
- Secondary eclipse search → `V02`
- Pixel centroid shift → `V08`
- Difference image localization → `V09`
- Aperture dependence → `V10`
- Model competition (transit vs non-transit alternatives) → `V16`
- Alias/phase diagnostics (phase-shift events) → `V19`

The markdown report exported above includes the full per-check provenance.
